In [1]:
#!/usr/bin/env python
import sys
import copy
#Opencv
import cv2 as cv
#from cv_bridge import CvBridge, CvBridgeError
import numpy as np
from numpy import mat
# ROS packages
import rospy
import tf
import time
import logging
#Robotiq package
from robotiq_ft_sensor.msg import ft_sensor
from robotiq_2f_gripper_control.msg import _Robotiq2FGripper_robot_output as outputMsg
from robotiq_2f_gripper_control.msg import _Robotiq2FGripper_robot_input  as inputMsg
from robotiq_ft_sensor.msg import ft_sensor
# from robotiq_c_model_control.msg import _CModel_robot_output as outputMsg
# ROS Image message
import geometry_msgs.msg
from std_msgs.msg import Int32
from geometry_msgs.msg import PoseStamped, Pose
from geometry_msgs.msg import PoseWithCovariance
from geometry_msgs.msg import WrenchStamped
from sensor_msgs.msg import Image
from std_msgs.msg import String, Empty, UInt16
#Math
import math
from math import pi, sin, cos, atan2
from math import sqrt, pi, acos, sin, cos
import math3d as m3d
#UR Control
import urx
from apriltag_ros.msg import AprilTagDetection, AprilTagDetectionArray

from tf.transformations import euler_from_quaternion, quaternion_from_euler
from a4_paper_turning.msg import pos
import actionlib
import xlwt

from robotiq_2f_gripper_msgs.msg import CommandRobotiqGripperFeedback, CommandRobotiqGripperResult, CommandRobotiqGripperAction, CommandRobotiqGripperGoal
from robotiq_2f_gripper_control.robotiq_2f_gripper_driver import Robotiq2FingerGripperDriver as Robotiq
from geometry_msgs.msg import WrenchStamped

In [2]:
#Connect to gripper
# gripper_pub = rospy.Publisher('Robotiq2FGripperRobotOutput', outputMsg.Robotiq2FGripper_robot_output, queue_size=10)
# arduino_pub1 = rospy.Publisher('/dac1', UInt16, queue_size=1)
# rospy.sleep(0.5)
# arduino_pub2 = rospy.Publisher('/dac2', UInt16, queue_size=1)
# rospy.sleep(0.5)
#Connect to UR robot
#robHong = urx.Robot("192.168.1.102")
arduino_pub1 = rospy.Publisher('/dac1', UInt16, queue_size=1)
arduino_pub2 = rospy.Publisher('/dac2', UInt16, queue_size=1)
servo_pub = rospy.Publisher('/servo', UInt16, queue_size=10)
rospy.sleep(0.5)
robKong = urx.Robot('192.168.1.10')
robHong = urx.Robot('192.168.1.102')
# rospy.init_node('sub_pub',anonymous=True)
rospy.init_node('picking_acrylic_board', anonymous=True)
logging.basicConfig(level=logging.WARN)
action_name = 'command_robotiq_action'
robotiq_client = actionlib.SimpleActionClient(action_name, CommandRobotiqGripperAction)
robotiq_client.wait_for_server()

timeout: timed out

In [4]:
def robKong_go_to_home():
    #1st valley fold, robkong flexes the paper

    Kong_joint0 = -pi*90.16/180
    Kong_joint1 = -pi*70.08/180#-pi*68.03/180 #81.05
    Kong_joint2 = pi*130.76/180#-pi*137.81/180 #128.91
    Kong_joint3 = -pi*150.66/180
    Kong_joint4 = -pi*89.97/180#pi*1/2#-pi*1/2
    Kong_joint5 = -pi*0.17/180
    robKong.movej((Kong_joint0,Kong_joint1, Kong_joint2, Kong_joint3, Kong_joint4, Kong_joint5), 0.5, 0.2)
def robKong_go_to_home1():
    #1st valley fold, robkong flexes the paper

    Kong_joint0 = -pi*78.65/180
    Kong_joint1 = -pi*68.25/180#-pi*68.03/180 #81.05
    Kong_joint2 = pi*118.22/180#-pi*137.81/180 #128.91
    Kong_joint3 = -pi*140.31/180
    Kong_joint4 = -pi*90.83/180#pi*1/2#-pi*1/2
    Kong_joint5 = -pi*84.31/180
    robKong.movej((Kong_joint0,Kong_joint1, Kong_joint2, Kong_joint3, Kong_joint4, Kong_joint5), 0.5, 0.2)
def robKong_go_to_home_mid():
    #1st valley fold, robkong flexes the paper

    Kong_joint0 = pi*80/180
    Kong_joint1 = -pi*78.88/180#-pi*68.03/180 #81.05
    Kong_joint2 = pi*126.77/180#-pi*137.81/180 #128.91
    Kong_joint3 = -pi*137.66/180
    Kong_joint4 = -pi*89.63/180#pi*1/2#-pi*1/2
    Kong_joint5 = -pi*55/180
    robKong.movej((Kong_joint0,Kong_joint1, Kong_joint2, Kong_joint3, Kong_joint4, Kong_joint5), 0.5, 0.2)
def robKong_go_to_corner1():
    #1st valley fold, robkong flexes the paper

    Kong_joint0 = pi*60.1/180
    Kong_joint1 = -pi*53.65/180#-pi*68.03/180 #81.05
    Kong_joint2 = pi*92.94/180#-pi*137.81/180 #128.91
    Kong_joint3 = -pi*128.92/180
    Kong_joint4 = -pi*89.97/180#pi*1/2#-pi*1/2
    Kong_joint5 = pi*99.39/180
    robKong.movej((Kong_joint0,Kong_joint1, Kong_joint2, Kong_joint3, Kong_joint4, Kong_joint5), 0.5, 0.2)
def robHong_go_to_home_mid():
    #1st valley fold, robkong flexes the paper

    Kong_joint0 = -pi*65.47/180
    Kong_joint1 = -pi*106.1/180#-pi*68.03/180 #81.05
    Kong_joint2 = -pi*103.22/180#-pi*137.81/180 #128.91
    Kong_joint3 = -pi*61.23/180
    Kong_joint4 = pi*88.19/180#pi*1/2#-pi*1/2
    Kong_joint5 = -pi*63.4/180
    robHong.movej((Kong_joint0,Kong_joint1, Kong_joint2, Kong_joint3, Kong_joint4, Kong_joint5), 0.5, 0.2)
def robHong_go_to_home_temp():
    #1st valley fold, robkong flexes the paper

    Kong_joint0 = -pi*72.99/180
    Kong_joint1 = -pi*115.23/180#-pi*68.03/180 #81.05
    Kong_joint2 = -pi*90.02/180#-pi*137.81/180 #128.91
    Kong_joint3 = -pi*65.15/180
    Kong_joint4 = pi*87.96/180#pi*1/2#-pi*1/2
    Kong_joint5 = -pi*160.96/180
    robHong.movej((Kong_joint0,Kong_joint1, Kong_joint2, Kong_joint3, Kong_joint4, Kong_joint5), 0.5, 0.2)

In [5]:
def move_ref_tcp(target_ref_transform, tcp_to_frame, acc=0.1, vel=0.02, wait=True):
    temp1 = m3d.Transform.get_inverse(m3d.Transform(tcp_to_frame))
    #print temp1
    temp2 = m3d.Transform(target_ref_transform)
    delta_rotation = temp1.orient.inverse * temp2.orient * temp1.orient
    delta_translation = temp2.pos + temp2.orient*temp1.pos - temp1.pos
    Delta_tcp = m3d.Transform()
    Delta_tcp.orient = delta_rotation
    Delta_tcp.pos = temp1.orient.inverse * delta_translation
    #print m3d.Transform(Delta_tcp)
    robKong.movel_tool(m3d.Transform.get_pose_vector(Delta_tcp), acc, vel, wait)
def move_ref_tcp_hong(target_ref_transform, tcp_to_frame, acc=0.1, vel=0.02, wait=True):
    temp1 = m3d.Transform.get_inverse(m3d.Transform(tcp_to_frame))
    #print temp1
    temp2 = m3d.Transform(target_ref_transform)
    delta_rotation = temp1.orient.inverse * temp2.orient * temp1.orient
    delta_translation = temp2.pos + temp2.orient*temp1.pos - temp1.pos
    Delta_tcp = m3d.Transform()
    Delta_tcp.orient = delta_rotation
    Delta_tcp.pos = temp1.orient.inverse * delta_translation
    #print m3d.Transform(Delta_tcp)
    robHong.movel_tool(m3d.Transform.get_pose_vector(Delta_tcp), acc, vel, wait)

In [6]:
robKong.set_tcp((0,0,0,0,0,0))
rospy.sleep(1)
tag_pose_x1 = 0
tag_pose_y1 = 0
tag_angle1 = 0
is_send_tag1=0
tag_topic1 = '/cam1/tag_detections'
#Robot motion speed percentage (0.5 ~ 3.0)

def kong_approach_the_object(a):

    global tag_pose_x1
    global tag_pose_y1
    global tag_angle1
    global is_send_tag1
    global tag_topic1
    is_send_tag1=int(a)    
    
def kong_detect_the_object(a):
    
    global tag_pose_x1
    global tag_pose_y1
    global tag_angle1
    global is_send_tag1
    global tag_topic1
    is_send_tag1=int(a)
    print('angle',tag_angle1/ 3.14159296 * 180)
       
    
def kong_tag_callback(msg):
    global is_send_tag1
    global tag_pose_x1
    global tag_pose_y1
    global tag_pose_z1
    global tag_angle1

    if is_send_tag1 == 1:
        tag_orientation1 = msg.detections[0].pose.pose.pose.orientation
        rpy = euler_from_quaternion([tag_orientation1.x, tag_orientation1.y, tag_orientation1.z, tag_orientation1.w])
        tag_angle1 = rpy[2]
        tag_pose_x1 = msg.detections[0].pose.pose.pose.position.x
        tag_pose_y1 = msg.detections[0].pose.pose.pose.position.y
        tag_pose_z1 = msg.detections[0].pose.pose.pose.position.z
        rospy.sleep(1)
        print 'tag_x = ', tag_pose_x1, ',tag_y = ', tag_pose_y1, ',tag_z- ',tag_pose_z1, ',tag angle = ', tag_angle1/ 3.14159296 * 180
        rospy.sleep(1)
        robKong.translate_tool((tag_pose_y1, -tag_pose_x1, 0), acc=0.1, vel=0.1, wait=True)
        is_send_tag1=0

    if is_send_tag1 == 2:
        tag_orientation1 = msg.detections[0].pose.pose.pose.orientation
        rpy = euler_from_quaternion([tag_orientation1.x, tag_orientation1.y, tag_orientation1.z, tag_orientation1.w])
        tag_angle1 = rpy[2]
        tag_pose_x1 = msg.detections[0].pose.pose.pose.position.x
        tag_pose_y1 = msg.detections[0].pose.pose.pose.position.y
        tag_pose_z1 = msg.detections[0].pose.pose.pose.position.z
        rospy.sleep(1)
        print 'tag_x = ', tag_pose_x1, ',tag_y = ', tag_pose_y1, ',tag_z- ',tag_pose_z1, ',tag angle = ', tag_angle1/ 3.14159296 * 180
        is_send_tag1=0

In [7]:
robHong.set_tcp((0,0,0,0,0,0))
rospy.sleep(1)
tag_pose_x = 0
tag_pose_y = 0
tag_angle = 0
is_send_tag=0
tag_topic2 = '/cam2/tag_detections'
#Robot motion speed percentage (0.5 ~ 3.0)

def hong_approach_the_object(a):

    global tag_pose_x
    global tag_pose_y
    global tag_angle
    global is_send_tag
    global tag_topic
    #robot Kong approaches the center point of the object
    #pose = robHong.getl()
    is_send_tag=int(a)
    
    
def hong_detect_the_object(a):
    
    global tag_pose_x
    global tag_pose_y
    global tag_angle
    global is_send_tag
    global tag_topic
    #robot Kong approaches the center point of the object
    #pose = robHong.getl()
    is_send_tag=int(a)
    print('angle',tag_angle/ 3.14159296 * 180)
       
    
def hong_tag_callback(msg):
    global is_send_tag
    global tag_pose_x
    global tag_pose_y
    global tag_pose_z
    global tag_angle

    if is_send_tag == 1:
        tag_orientation = msg.detections[0].pose.pose.pose.orientation
        rpy = euler_from_quaternion([tag_orientation.x, tag_orientation.y, tag_orientation.z, tag_orientation.w])
        tag_angle = rpy[2]
        tag_pose_x = msg.detections[0].pose.pose.pose.position.x
        tag_pose_y = msg.detections[0].pose.pose.pose.position.y
        tag_pose_z = msg.detections[0].pose.pose.pose.position.z
        rospy.sleep(1)
        print 'tag_x = ', tag_pose_x, ',tag_y = ', tag_pose_y, ',tag_z- ',tag_pose_z, ',tag angle = ', tag_angle/ 3.14159296 * 180
        rospy.sleep(1)
        robHong.translate_tool((tag_pose_y, -tag_pose_x, 0), acc=0.1, vel=0.1, wait=True)
        is_send_tag=0

    if is_send_tag == 2:
        tag_orientation = msg.detections[0].pose.pose.pose.orientation
        rpy = euler_from_quaternion([tag_orientation.x, tag_orientation.y, tag_orientation.z, tag_orientation.w])
        tag_angle = rpy[2]
        tag_pose_x = msg.detections[0].pose.pose.pose.position.x
        tag_pose_y = msg.detections[0].pose.pose.pose.position.y
        tag_pose_z = msg.detections[0].pose.pose.pose.position.z
        rospy.sleep(1)
        print 'tag_x = ', tag_pose_x, ',tag_y = ', tag_pose_y, ',tag_z- ',tag_pose_z, ',tag angle = ', tag_angle/ 3.14159296 * 180
        is_send_tag=0

In [8]:
def move_to_pregrasp_posotion(dx, dy, dz):
    robKong.translate_tool((0.116 + dx,-0.085 + dy,0),acc=0.05,vel=0.05,wait=True)
    rospy.sleep(2)
    pos = robKong.get_pos()
    print('pos.z',pos.z)
    robKong.translate_tool((0,0,pos.z),acc=0.1,vel=0.1,wait=True)
    rospy.sleep(1)
    arduino_pub2.publish(800)
    rospy.sleep(1)
    robKong.translate_tool((0,0,0.078 + dz), acc=0.05, vel=0.05, wait=True)    

In [9]:
rospy.Subscriber(tag_topic1, AprilTagDetectionArray, kong_tag_callback, queue_size=10)

In [10]:
rospy.Subscriber(tag_topic2, AprilTagDetectionArray, hong_tag_callback, queue_size=10)

In [11]:
#kong_step 1: go to home and set the tip of underactuated finger as tcp
robKong_go_to_home_mid()
rospy.sleep(0.5)
robKong.set_tcp((0,0,0.14,0,0,0)) #z as unactuated soft finger
rospy.sleep(0.5)

In [13]:
#hong_step 1: go to the home position and set the tip of hold finger as tcp position (open:30)
robHong_go_to_home_mid()
rospy.sleep(1)
robHong.set_tcp((0.075,0,0.323,0,0,0))

In [20]:
#hong_step 2: rotate to 0 degree
for i in range(2):
    hong_detect_the_object(2)
    rospy.sleep(2)
    move_ref_tcp_hong((0,0,0,0,0,tag_angle),(0,0,0,0,0,0),acc=0.05,vel=0.05,wait=True)
    rospy.sleep(2)

('angle', -90.21878603390203)
tag_x =  -0.06860533599 ,tag_y =  -0.0602004713903 ,tag_z-  0.347299420981 ,tag angle =  -0.452249669965


In [21]:
#hong_step 3: detect the crease 
hong_approach_the_object(1)
rospy.sleep(2)
while (abs(tag_pose_x)>=0.001 or abs(tag_pose_y)>=0.001):
    hong_approach_the_object(1)
    rospy.sleep(3)
    print "1"

tag_x =  -0.0686545037794 ,tag_y =  -0.0605951028263 ,tag_z-  0.346854078012 ,tag angle =  -0.0528931441061
1
tag_x =  -0.000101711049552 ,tag_y =  0.00163706722035 ,tag_z-  0.349069893072 ,tag angle =  0.603041386343
1
tag_x =  -4.05909785853e-05 ,tag_y =  0.000142675572829 ,tag_z-  0.347736723773 ,tag angle =  0.402972333641
1


In [22]:
#hong_step 4:go to the fix position
robHong.translate_tool((0.2,-0.1,0),acc=0.05,vel=0.05,wait=True)
rospy.sleep(1)
move_ref_tcp_hong((0,0,0,pi/180,0,0),(0,0,0,0,0,0),acc=0.05,vel=0.05,wait=True)
rospy.sleep(1)
robHong.translate((0,0,-0.115),acc=0.05,vel=0.05,wait=True)

<Transform:
<Orientation: 
array([[ 0.99973322, -0.01101776, -0.02030027],
       [-0.01067862, -0.99980286,  0.01673957],
       [-0.0204807 , -0.01651832, -0.99965378]])>
<Vector: (0.30175, -0.87477, -0.09797)>
>

In [23]:
#Kong_step2: rotate to 45 degree
for i in range(2):
    kong_detect_the_object(2)
    rospy.sleep(2)
    move_ref_tcp((0,0,0,0,0,tag_angle1-pi/4),(0,0,0,0,0,0),acc=0.05,vel=0.05,wait=True)
    rospy.sleep(2)

('angle', 44.952385546084436)
tag_x =  -0.0104301038493 ,tag_y =  0.0194801893098 ,tag_z-  0.262842535555 ,tag angle =  45.6022868891
('angle', 45.602286889096526)
tag_x =  -0.00960454155553 ,tag_y =  0.0194774209286 ,tag_z-  0.262912058101 ,tag angle =  44.9927133902


In [24]:
#Kong_step 3: approach to the object
kong_approach_the_object(1)
rospy.sleep(1)
while (abs(tag_pose_x1)>=0.001 or abs(tag_pose_y1)>=0.001):
    kong_approach_the_object(1)
    rospy.sleep(3)
    print "1"

tag_x =  -0.00961204366711 ,tag_y =  0.0194735083967 ,tag_z-  0.262944704123 ,tag angle =  45.0265786711
1
tag_x =  -0.000180481456065 ,tag_y =  0.000128635388439 ,tag_z-  0.263378263158 ,tag angle =  44.6294059126
1


In [25]:
#kong_step 4: move to the pregrasp position
move_to_pregrasp_posotion(0.01,0,0.004)

('pos.z', 0.07214806819450947)


In [26]:
#kong_step 5:flex-flip and loosen finger and rotate 45 to align frame {O} and frame {G}
arduino_pub1.publish(3800)
rospy.sleep(1)
arduino_pub1.publish(1600)
rospy.sleep(2)
robKong.translate_tool((0,0,-0.02),acc=0.05,vel=0.05,wait=True)
rospy.sleep(1)
robKong.set_tcp((-0.02,0.08,0.09,0,0,0))
rospy.sleep(1)
move_ref_tcp((0,0,0,0,0,45*pi/180),(0,0,0,0,0,0), acc=0.05,vel=0.05,wait=True)

In [27]:
#kong_step 6: grap at the mid 
robKong.translate_tool((0,0,0.01),acc=0.05,vel=0.05,wait=True)
rospy.sleep(1)
robKong.translate_tool((0,-0.02,0),acc=0.05,vel=0.05,wait=True)
rospy.sleep(1)
robKong.translate_tool((0,0,-0.05),acc=0.05,vel=0.05,wait=True)
rospy.sleep(1)
arduino_pub1.publish(2600)
rospy.sleep(0.5)
arduino_pub2.publish(2600)

<Transform:
<Orientation: 
array([[-0.99977682,  0.01800161,  0.01105715],
       [ 0.01802594,  0.9998353 ,  0.00210518],
       [-0.01101743,  0.00230403, -0.99993665]])>
<Vector: (-0.03342, -0.75425, 0.02749)>
>

In [36]:
#kong_step 7: detect the crease and move to the target position
robKong.set_tcp((-0.018,0.08,0.133,0,0,0)) # set the actuated (2600) fingertip as tcp
rospy.sleep(1)
pos = robKong.get_pos()
print('x',pos.x)
print('y',pos.y)
kong_detect_the_object(2)
print('angle',tag_angle1)
y=tag_pose_y1+0.04+0.02+0.1285 #0.056:distance of cam to fingertip, 0.04: dis of fingertip to paper edge, 0.1285:half length of paper
x=tag_pose_x1
print("y",y)
print("x",x)

('x', -0.03472897847385084)
('y', -0.739064783499325)
('angle', -0.6479561401951257)
('angle', -0.011308969157921)
('y', 0.2100864235526332)
('x', 0.013252036450927096)


[ERROR] [1602510318.532701]: bad callback: <function kong_tag_callback at 0x7f062c0063d0>
Traceback (most recent call last):
  File "/opt/ros/kinetic/lib/python2.7/dist-packages/rospy/topics.py", line 750, in _invoke_callback
    cb(msg)
  File "<ipython-input-6-713fca2bd3cc>", line 51, in kong_tag_callback
    tag_orientation1 = msg.detections[0].pose.pose.pose.orientation
IndexError: list index out of range

[ERROR] [1602510318.581865]: bad callback: <function kong_tag_callback at 0x7f062c0063d0>
Traceback (most recent call last):
  File "/opt/ros/kinetic/lib/python2.7/dist-packages/rospy/topics.py", line 750, in _invoke_callback
    cb(msg)
  File "<ipython-input-6-713fca2bd3cc>", line 51, in kong_tag_callback
    tag_orientation1 = msg.detections[0].pose.pose.pose.orientation
IndexError: list index out of range

[ERROR] [1602510318.618278]: bad callback: <function kong_tag_callback at 0x7f062c0063d0>
Traceback (most recent call last):
  File "/opt/ros/kinetic/lib/python2.7/dist-pac

[ERROR] [1602510319.232979]: bad callback: <function kong_tag_callback at 0x7f062c0063d0>
Traceback (most recent call last):
  File "/opt/ros/kinetic/lib/python2.7/dist-packages/rospy/topics.py", line 750, in _invoke_callback
    cb(msg)
  File "<ipython-input-6-713fca2bd3cc>", line 51, in kong_tag_callback
    tag_orientation1 = msg.detections[0].pose.pose.pose.orientation
IndexError: list index out of range

[ERROR] [1602510319.267162]: bad callback: <function kong_tag_callback at 0x7f062c0063d0>
Traceback (most recent call last):
  File "/opt/ros/kinetic/lib/python2.7/dist-packages/rospy/topics.py", line 750, in _invoke_callback
    cb(msg)
  File "<ipython-input-6-713fca2bd3cc>", line 51, in kong_tag_callback
    tag_orientation1 = msg.detections[0].pose.pose.pose.orientation
IndexError: list index out of range

[ERROR] [1602510319.301079]: bad callback: <function kong_tag_callback at 0x7f062c0063d0>
Traceback (most recent call last):
  File "/opt/ros/kinetic/lib/python2.7/dist-pac

[ERROR] [1602510319.901413]: bad callback: <function kong_tag_callback at 0x7f062c0063d0>
Traceback (most recent call last):
  File "/opt/ros/kinetic/lib/python2.7/dist-packages/rospy/topics.py", line 750, in _invoke_callback
    cb(msg)
  File "<ipython-input-6-713fca2bd3cc>", line 51, in kong_tag_callback
    tag_orientation1 = msg.detections[0].pose.pose.pose.orientation
IndexError: list index out of range

[ERROR] [1602510319.933996]: bad callback: <function kong_tag_callback at 0x7f062c0063d0>
Traceback (most recent call last):
  File "/opt/ros/kinetic/lib/python2.7/dist-packages/rospy/topics.py", line 750, in _invoke_callback
    cb(msg)
  File "<ipython-input-6-713fca2bd3cc>", line 51, in kong_tag_callback
    tag_orientation1 = msg.detections[0].pose.pose.pose.orientation
IndexError: list index out of range

[ERROR] [1602510319.966771]: bad callback: <function kong_tag_callback at 0x7f062c0063d0>
Traceback (most recent call last):
  File "/opt/ros/kinetic/lib/python2.7/dist-pac

[ERROR] [1602510320.739497]: bad callback: <function kong_tag_callback at 0x7f062c0063d0>
Traceback (most recent call last):
  File "/opt/ros/kinetic/lib/python2.7/dist-packages/rospy/topics.py", line 750, in _invoke_callback
    cb(msg)
  File "<ipython-input-6-713fca2bd3cc>", line 51, in kong_tag_callback
    tag_orientation1 = msg.detections[0].pose.pose.pose.orientation
IndexError: list index out of range

[ERROR] [1602510320.772190]: bad callback: <function kong_tag_callback at 0x7f062c0063d0>
Traceback (most recent call last):
  File "/opt/ros/kinetic/lib/python2.7/dist-packages/rospy/topics.py", line 750, in _invoke_callback
    cb(msg)
  File "<ipython-input-6-713fca2bd3cc>", line 51, in kong_tag_callback
    tag_orientation1 = msg.detections[0].pose.pose.pose.orientation
IndexError: list index out of range

[ERROR] [1602510320.807134]: bad callback: <function kong_tag_callback at 0x7f062c0063d0>
Traceback (most recent call last):
  File "/opt/ros/kinetic/lib/python2.7/dist-pac

...

[ERROR] [1602510327.266879]: bad callback: <function kong_tag_callback at 0x7f062c0063d0>
Traceback (most recent call last):
  File "/opt/ros/kinetic/lib/python2.7/dist-packages/rospy/topics.py", line 750, in _invoke_callback
    cb(msg)
  File "<ipython-input-6-713fca2bd3cc>", line 51, in kong_tag_callback
    tag_orientation1 = msg.detections[0].pose.pose.pose.orientation
IndexError: list index out of range

[ERROR] [1602510327.296574]: bad callback: <function kong_tag_callback at 0x7f062c0063d0>
Traceback (most recent call last):
  File "/opt/ros/kinetic/lib/python2.7/dist-packages/rospy/topics.py", line 750, in _invoke_callback
    cb(msg)
  File "<ipython-input-6-713fca2bd3cc>", line 51, in kong_tag_callback
    tag_orientation1 = msg.detections[0].pose.pose.pose.orientation
IndexError: list index out of range

[ERROR] [1602510327.330598]: bad callback: <function kong_tag_callback at 0x7f062c0063d0>
Traceback (most recent call last):
  File "/opt/ros/kinetic/lib/python2.7/dist-pac

[ERROR] [1602510328.135336]: bad callback: <function kong_tag_callback at 0x7f062c0063d0>
Traceback (most recent call last):
  File "/opt/ros/kinetic/lib/python2.7/dist-packages/rospy/topics.py", line 750, in _invoke_callback
    cb(msg)
  File "<ipython-input-6-713fca2bd3cc>", line 51, in kong_tag_callback
    tag_orientation1 = msg.detections[0].pose.pose.pose.orientation
IndexError: list index out of range

[ERROR] [1602510328.171499]: bad callback: <function kong_tag_callback at 0x7f062c0063d0>
Traceback (most recent call last):
  File "/opt/ros/kinetic/lib/python2.7/dist-packages/rospy/topics.py", line 750, in _invoke_callback
    cb(msg)
  File "<ipython-input-6-713fca2bd3cc>", line 51, in kong_tag_callback
    tag_orientation1 = msg.detections[0].pose.pose.pose.orientation
IndexError: list index out of range

[ERROR] [1602510328.201620]: bad callback: <function kong_tag_callback at 0x7f062c0063d0>
Traceback (most recent call last):
  File "/opt/ros/kinetic/lib/python2.7/dist-pac

[ERROR] [1602510328.805664]: bad callback: <function kong_tag_callback at 0x7f062c0063d0>
Traceback (most recent call last):
  File "/opt/ros/kinetic/lib/python2.7/dist-packages/rospy/topics.py", line 750, in _invoke_callback
    cb(msg)
  File "<ipython-input-6-713fca2bd3cc>", line 51, in kong_tag_callback
    tag_orientation1 = msg.detections[0].pose.pose.pose.orientation
IndexError: list index out of range

[ERROR] [1602510328.838759]: bad callback: <function kong_tag_callback at 0x7f062c0063d0>
Traceback (most recent call last):
  File "/opt/ros/kinetic/lib/python2.7/dist-packages/rospy/topics.py", line 750, in _invoke_callback
    cb(msg)
  File "<ipython-input-6-713fca2bd3cc>", line 51, in kong_tag_callback
    tag_orientation1 = msg.detections[0].pose.pose.pose.orientation
IndexError: list index out of range

[ERROR] [1602510328.869825]: bad callback: <function kong_tag_callback at 0x7f062c0063d0>
Traceback (most recent call last):
  File "/opt/ros/kinetic/lib/python2.7/dist-pac

[ERROR] [1602510329.508225]: bad callback: <function kong_tag_callback at 0x7f062c0063d0>
Traceback (most recent call last):
  File "/opt/ros/kinetic/lib/python2.7/dist-packages/rospy/topics.py", line 750, in _invoke_callback
    cb(msg)
  File "<ipython-input-6-713fca2bd3cc>", line 51, in kong_tag_callback
    tag_orientation1 = msg.detections[0].pose.pose.pose.orientation
IndexError: list index out of range

[ERROR] [1602510329.542569]: bad callback: <function kong_tag_callback at 0x7f062c0063d0>
Traceback (most recent call last):
  File "/opt/ros/kinetic/lib/python2.7/dist-packages/rospy/topics.py", line 750, in _invoke_callback
    cb(msg)
  File "<ipython-input-6-713fca2bd3cc>", line 51, in kong_tag_callback
    tag_orientation1 = msg.detections[0].pose.pose.pose.orientation
IndexError: list index out of range

[ERROR] [1602510329.574857]: bad callback: <function kong_tag_callback at 0x7f062c0063d0>
Traceback (most recent call last):
  File "/opt/ros/kinetic/lib/python2.7/dist-pac

tag_x =  0.0147822020785 ,tag_y =  0.0259280520057 ,tag_z-  0.149404125135 ,tag angle =  4.93494191618


In [35]:
robKong.translate_tool((-y,0,0),acc=0.05,vel=0.05,wait=True)

<Transform:
<Orientation: 
array([[-0.99978005,  0.01781915,  0.01105999],
       [ 0.01784456,  0.99983835,  0.00220238],
       [-0.01101896,  0.00239926, -0.99993641]])>
<Vector: (-0.03483, -0.73907, -0.01533)>
>

In [31]:
kong_detect_the_object(2)

('angle', 44.62940591259952)
tag_x =  0.0138615481338 ,tag_y =  0.015446613186 ,tag_z-  0.164434644906 ,tag angle =  -0.0360055448401


In [128]:
robHong.set_tcp((0.08,0,0.315,0,0,0))
rospy.sleep(1)
robHong_go_to_home_temp()

In [62]:
move_ref_tcp_hong((0,0,0,pi/180,0,0),(0,0,0,0,0,0),acc=0.05,vel=0.05,wait=True)

In [63]:
robHong.translate_tool((0,0.06,0),acc=0.05,vel=0.05,wait=True)

<Transform:
<Orientation: 
array([[ 0.99953102, -0.02313399, -0.02006375],
       [-0.02242132, -0.99913415,  0.0350463 ],
       [-0.02085714, -0.03458001, -0.99918427]])>
<Vector: (0.18750, -0.96224, 0.02443)>
>

In [66]:
robHong.translate_tool((0,0,0.095),acc=0.05,vel=0.05,wait=True)

<Transform:
<Orientation: 
array([[ 0.99954001, -0.02307744, -0.01967721],
       [-0.02239369, -0.99916107,  0.03428809],
       [-0.02045199, -0.03383167, -0.99921826]])>
<Vector: (0.18579, -0.95928, -0.06982)>
>

('pos.z', 0.07277722413857837)


<Transform:
<Orientation: 
array([[-0.68729873, -0.72629588,  0.01071222],
       [-0.72631932,  0.68735381,  0.00223079],
       [-0.0089833 , -0.00624727, -0.99994013]])>
<Vector: (0.06000, -0.71744, -0.06290)>
>

In [114]:
robHong.translate_tool((0,0,-0.05),acc=0.05,vel=0.05,wait=True)

<Transform:
<Orientation: 
array([[ 0.9995266 , -0.02302889, -0.02040202],
       [-0.02229171, -0.99911562,  0.03565178],
       [-0.021205  , -0.03518011, -0.999156  ]])>
<Vector: (0.18637, -0.96056, -0.02104)>
>

In [115]:
robHong.translate_tool((0,-0.06,0),acc=0.05,vel=0.05,wait=True)

<Transform:
<Orientation: 
array([[ 0.99952603, -0.023033  , -0.02042527],
       [-0.02229497, -0.99911551,  0.03565304],
       [-0.0212284 , -0.03518076, -0.99915548]])>
<Vector: (0.18773, -0.90067, -0.01869)>
>

In [116]:
robHong.translate_tool((-0.08,0,0),acc=0.05,vel=0.05,wait=True)

<Transform:
<Orientation: 
array([[ 0.9995208 , -0.02331126, -0.02036546],
       [-0.02257626, -0.99911095,  0.03560381],
       [-0.02117732, -0.03512697, -0.99915845]])>
<Vector: (0.10791, -0.89885, -0.01700)>
>

In [117]:
robHong.translate_tool((0,0.05,0),acc=0.05,vel=0.05,wait=True)

<Transform:
<Orientation: 
array([[ 0.99952932, -0.02307942, -0.02021079],
       [-0.02235586, -0.99912586,  0.03532323],
       [-0.02100836, -0.03485477, -0.99917155]])>
<Vector: (0.10682, -0.94883, -0.01863)>
>

In [120]:
robKong.translate_tool((0.21,0,0),acc=0.05,vel=0.05,wait=True)

<Transform:
<Orientation: 
array([[-0.99993801, -0.0034421 ,  0.01058911],
       [-0.0034209 ,  0.99999211,  0.00201996],
       [-0.01059598,  0.00198361, -0.99994189]])>
<Vector: (-0.25586, -0.66083, 0.04548)>
>

In [121]:
robKong.translate_tool((0,0,0.075),acc=0.05,vel=0.05,wait=True)

<Transform:
<Orientation: 
array([[-0.99993902, -0.0035343 ,  0.01046302],
       [-0.00351252,  0.99999163,  0.00209934],
       [-0.01047036,  0.00206246, -0.99994306]])>
<Vector: (-0.25524, -0.66067, -0.02944)>
>

In [122]:
robHong.translate_tool((0,0,-0.055),acc=0.05,vel=0.05,wait=True)

<Transform:
<Orientation: 
array([[ 0.99954288, -0.02297888, -0.01964718],
       [-0.02234823, -0.99924637,  0.03173718],
       [-0.02036165, -0.03128359, -0.99930313]])>
<Vector: (0.10656, -0.94925, -0.02023)>
>

In [123]:
move_ref_tcp_hong((0,0,0,0,0,pi/2),(0,0,0,0,0,0),acc=0.05,vel=0.05,wait=True)

In [124]:
robHong.set_tcp((-0.068,0,0.31,0,0,0))
rospy.sleep(1)
move_ref_tcp_hong((0,0,0,0,20*pi/180,0),(0,0,0,0,0,0),acc=0.05,vel=0.05,wait=True)

In [125]:
robHong.translate((0,-0.4,0),acc=0.05,vel=0.05,wait=True)

<Transform:
<Orientation: 
array([[-0.01497815, -0.99954028, -0.0263607 ],
       [-0.94978705,  0.02246328, -0.31208966],
       [ 0.31253833,  0.02036252, -0.94968687]])>
<Vector: (0.11002, -1.20161, -0.01028)>
>

In [126]:
robHong.translate((0,0,-0.063),acc=0.05,vel=0.05,wait=True)

<Transform:
<Orientation: 
array([[-0.01502293, -0.99953977, -0.02635468],
       [-0.94982537,  0.02250139, -0.31197028],
       [ 0.31241972,  0.02034563, -0.94972626]])>
<Vector: (0.10984, -1.20166, -0.07314)>
>

In [127]:
robHong.translate((0,0.18,0),acc=0.05,vel=0.05,wait=True)

<Transform:
<Orientation: 
array([[-0.01502296, -0.99953731, -0.02644771],
       [-0.94991027,  0.0225249 , -0.31170998],
       [ 0.31216149,  0.02044015, -0.94980914]])>
<Vector: (0.10974, -1.02164, -0.07350)>
>